<a href="https://colab.research.google.com/github/eshal26/PCA-CNN/blob/main/VGG_increasedim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os
import shutil
import sklearn
from sklearn.model_selection import train_test_split

In [4]:
train_dir = 'train_dataset'
val_dir = 'validation_dataset'
test_dir = 'test_dataset'

# Define transformations for training, validation, and testing data
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.8238, 0.8539, 0.9391], std=[0.1325, 0.1437, 0.0529])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.8238, 0.8539, 0.9391], std=[0.1325, 0.1437, 0.0529])
])

# Create datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_test_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=val_test_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [7]:
from sklearn.decomposition import PCA
import torch
import torch.nn as nn

class CustomVGGWithPCA(nn.Module):
    def __init__(self, num_classes=2, pca_components=512):
        super(CustomVGGWithPCA, self).__init__()

        # Feature extraction layers with more filters
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 112x112x256

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 56x56x512

            # Removed additional pooling layers to keep more features
        )

        # Initialize PCA to None
        self.pca = None

        # Classifier layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(pca_components, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, num_classes)
        )

    def extract_features(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the features for PCA
        return x



    def forward(self, x):
        x = self.extract_features(x)
        device = x.device

        # Apply PCA if it is fitted
        if self.pca is not None:
            # Convert tensor to NumPy array (detach and move to CPU)
            x_cpu = x.detach().cpu().numpy()

            # Apply PCA transformation
            x_pca = self.pca.transform(x_cpu)  # Apply PCA transformation on NumPy array

            # Add padding or random noise to increase dimensions
            if x_pca.shape[1] < self.pca.components_.shape[0]:  # Check if the PCA components are larger
                padding = torch.zeros((x_pca.shape[0], self.pca.components_.shape[0] - x_pca.shape[1]))
                x_pca = torch.cat((torch.tensor(x_pca), padding), dim=1)

            # Convert the transformed data back to PyTorch tensor
            x = torch.from_numpy(x_pca).to(device, dtype=torch.float32)
        else:
            raise RuntimeError("PCA must be fitted before the forward pass")

        x = self.classifier(x)
        return x

def fit_pca_on_train_data(train_loader, model, device, pca_components ,  batch_size=64):
    with torch.no_grad():
        sample_batch, _ = next(iter(train_loader))
        sample_features = model.extract_features(sample_batch.to(device))
        num_features = sample_features.shape[1]  # Get feature dimension from sample batch

    ipca = PCA(n_components=pca_components)

    for images, _ in train_loader:
        images = images.to(device)
        with torch.no_grad():
            features = model.extract_features(images)
            features = features.detach().cpu().numpy()
            ipca.fit(features)  # Fit the PCA to the data

    model.pca = ipca
    print("PCA fitting completed on training data.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pca_components = 512  # Set higher components to increase dimensions
num_classes = 2

model = CustomVGGWithPCA(num_classes=num_classes, pca_components=pca_components).to(device)
fit_pca_on_train_data(train_loader,  model,  device, pca_components=pca_components)


ValueError: n_components=512 must be between 0 and min(n_samples, n_features)=64 with svd_solver='full'

In [ ]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct_predictions, total_samples = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = 100 * correct_predictions / total_samples
        print(f"Epoch {epoch+1}/{num_epochs}: Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=40)
